## Unmasking

### Ingredients:


In [133]:
from sklearn.svm import LinearSVC, SVC
import numpy as np
import pickle
import sys
import os
import warnings
import pickle
from sklearn.exceptions import ConvergenceWarning
import torch
from dataset_readers.mini_dataset import MiniDataset
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_validate
from sklearn.metrics import roc_curve, roc_auc_score

In [70]:
# same_pairs, diff_pairs = pickle.load(open('./selected_training_samples.pkl', 'rb'))
# same_accs = []
# for pair in diff_pairs:
#     dataset = np.concatenate(pair['pair_emb'], axis = 0)
#     labels = np.array([0] * len(pair['pair_emb'][0]) + [1] * len(pair['pair_emb'][1]))

#     accs = []
#     for _ in range(20):
#         model = LinearSVC()
#         model.fit(dataset, labels)
#         loss = np.mean(model.predict(dataset) == labels)
#         accs.append(loss)

#         features = np.argsort(np.abs(model.coef_))[0][::-1]
#         dataset = np.delete(dataset, features[:20], 1)
#     same_accs.append(accs)
# pickle.dump(same_accs, open('./diff_accs.pkl', 'wb'))

In [93]:
mini_ds_paths = {
    "train": "/pan2020/open_splits/unseen_authors/xs/pan20-av-small-train/unmasking",
    "val": "/pan2020/open_splits/unseen_authors/xs/pan20-av-small-val/unmasking",
    "test": "/pan2020/open_splits/unseen_authors/xs/pan20-av-small-test/unmasking"
}
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    train_curves = []
    train_labels = []
    for idx, ds_fname in enumerate(os.listdir(mini_ds_paths['train'])):
        print("Processed %d train mini-datasets " %(idx))
        # a mini-dataset contains examples generated from two documents X and Y
        # specifically, X and Y are split into equal size chunks, and the 
        # resulting chunks become the training examples for the unmasking method
        mini_dataset_path = os.path.join(mini_ds_paths['train'], ds_fname)
        mini_dataset = torch.load(mini_dataset_path)
        #print(mini_dataset)
        data, labels = mini_dataset.get_tensor_dataset()
        data = data.cpu().numpy()
        labels = labels.numpy()

        ds_accs = []
        for round in range(20):
            # measure cross-validation accuracy using a LinearSVC over sequence embedding
            model = LinearSVC(max_iter=2000)    
            results = cross_validate(
                estimator=model, 
                X=data,
                y=labels,
                scoring='accuracy',
                cv=5,
                return_estimator=True
            )
            acc = np.mean(results['test_score'])
            #print("Round %d, cross-val acc %f" % (round, acc))
            ds_accs.append(acc.item())

            # 5 x num_features
            weights = np.concatenate([est.coef_ for est in results['estimator']])

            # for each weight, compute its absolute max value across models
            # then take the top 20 maximum weights to indicate the most
            # important features
            # num_features
            max_weights = np.max(np.abs(weights), axis=0)
            # feature indices from most important to least important
            # num_features
            features_idx = np.argsort(max_weights)[::-1]

            # eliminate features from dataset
            data = np.delete(data, features_idx[:20], 1)
            #print("   data shape: ", data.shape)

        #print("label = ", mini_dataset.pair_label, ", acc = ", ds_accs)
        train_curves.append(ds_accs)
        train_labels.append(mini_dataset.pair_label)
        #if idx == 5:
        #    break

Processed 0 train mini-datasets 
Processed 1 train mini-datasets 
Processed 2 train mini-datasets 
Processed 3 train mini-datasets 
Processed 4 train mini-datasets 
Processed 5 train mini-datasets 
Processed 6 train mini-datasets 
Processed 7 train mini-datasets 
Processed 8 train mini-datasets 
Processed 9 train mini-datasets 
Processed 10 train mini-datasets 
Processed 11 train mini-datasets 
Processed 12 train mini-datasets 
Processed 13 train mini-datasets 
Processed 14 train mini-datasets 
Processed 15 train mini-datasets 
Processed 16 train mini-datasets 
Processed 17 train mini-datasets 
Processed 18 train mini-datasets 
Processed 19 train mini-datasets 
Processed 20 train mini-datasets 
Processed 21 train mini-datasets 
Processed 22 train mini-datasets 
Processed 23 train mini-datasets 
Processed 24 train mini-datasets 
Processed 25 train mini-datasets 
Processed 26 train mini-datasets 
Processed 27 train mini-datasets 
Processed 28 train mini-datasets 
Processed 29 train mini-

Processed 238 train mini-datasets 
Processed 239 train mini-datasets 
Processed 240 train mini-datasets 
Processed 241 train mini-datasets 
Processed 242 train mini-datasets 
Processed 243 train mini-datasets 
Processed 244 train mini-datasets 
Processed 245 train mini-datasets 
Processed 246 train mini-datasets 
Processed 247 train mini-datasets 
Processed 248 train mini-datasets 
Processed 249 train mini-datasets 
Processed 250 train mini-datasets 
Processed 251 train mini-datasets 
Processed 252 train mini-datasets 
Processed 253 train mini-datasets 
Processed 254 train mini-datasets 
Processed 255 train mini-datasets 
Processed 256 train mini-datasets 
Processed 257 train mini-datasets 
Processed 258 train mini-datasets 
Processed 259 train mini-datasets 
Processed 260 train mini-datasets 
Processed 261 train mini-datasets 
Processed 262 train mini-datasets 
Processed 263 train mini-datasets 
Processed 264 train mini-datasets 
Processed 265 train mini-datasets 
Processed 266 train 

Processed 473 train mini-datasets 
Processed 474 train mini-datasets 
Processed 475 train mini-datasets 
Processed 476 train mini-datasets 
Processed 477 train mini-datasets 
Processed 478 train mini-datasets 
Processed 479 train mini-datasets 
Processed 480 train mini-datasets 
Processed 481 train mini-datasets 
Processed 482 train mini-datasets 
Processed 483 train mini-datasets 
Processed 484 train mini-datasets 
Processed 485 train mini-datasets 
Processed 486 train mini-datasets 
Processed 487 train mini-datasets 
Processed 488 train mini-datasets 
Processed 489 train mini-datasets 
Processed 490 train mini-datasets 
Processed 491 train mini-datasets 
Processed 492 train mini-datasets 
Processed 493 train mini-datasets 
Processed 494 train mini-datasets 
Processed 495 train mini-datasets 
Processed 496 train mini-datasets 
Processed 497 train mini-datasets 
Processed 498 train mini-datasets 
Processed 499 train mini-datasets 
Processed 500 train mini-datasets 
Processed 501 train 

Processed 708 train mini-datasets 
Processed 709 train mini-datasets 
Processed 710 train mini-datasets 
Processed 711 train mini-datasets 
Processed 712 train mini-datasets 
Processed 713 train mini-datasets 
Processed 714 train mini-datasets 
Processed 715 train mini-datasets 
Processed 716 train mini-datasets 
Processed 717 train mini-datasets 
Processed 718 train mini-datasets 
Processed 719 train mini-datasets 
Processed 720 train mini-datasets 
Processed 721 train mini-datasets 
Processed 722 train mini-datasets 
Processed 723 train mini-datasets 
Processed 724 train mini-datasets 
Processed 725 train mini-datasets 
Processed 726 train mini-datasets 
Processed 727 train mini-datasets 
Processed 728 train mini-datasets 
Processed 729 train mini-datasets 
Processed 730 train mini-datasets 
Processed 731 train mini-datasets 
Processed 732 train mini-datasets 
Processed 733 train mini-datasets 
Processed 734 train mini-datasets 
Processed 735 train mini-datasets 
Processed 736 train 

Processed 943 train mini-datasets 
Processed 944 train mini-datasets 
Processed 945 train mini-datasets 
Processed 946 train mini-datasets 
Processed 947 train mini-datasets 
Processed 948 train mini-datasets 
Processed 949 train mini-datasets 
Processed 950 train mini-datasets 
Processed 951 train mini-datasets 
Processed 952 train mini-datasets 
Processed 953 train mini-datasets 
Processed 954 train mini-datasets 
Processed 955 train mini-datasets 
Processed 956 train mini-datasets 
Processed 957 train mini-datasets 
Processed 958 train mini-datasets 
Processed 959 train mini-datasets 
Processed 960 train mini-datasets 
Processed 961 train mini-datasets 
Processed 962 train mini-datasets 
Processed 963 train mini-datasets 
Processed 964 train mini-datasets 
Processed 965 train mini-datasets 
Processed 966 train mini-datasets 
Processed 967 train mini-datasets 
Processed 968 train mini-datasets 
Processed 969 train mini-datasets 
Processed 970 train mini-datasets 
Processed 971 train 

In [94]:
# save training curves
with open('train_curves.pickle', 'wb') as fp:
    pickle.dump((np.array(train_curves), np.array(train_labels)), fp)

In [97]:
# COMPUTE TEST CURVES
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    test_curves = []
    test_labels = []
    for idx, ds_fname in enumerate(os.listdir(mini_ds_paths['test'])):
        print("Processed %d test mini-datasets " %(idx))
        # a mini-dataset contains examples generated from two documents X and Y
        # specifically, X and Y are split into equal size chunks, and the 
        # resulting chunks become the training examples for the unmasking method
        mini_dataset_path = os.path.join(mini_ds_paths['test'], ds_fname)
        mini_dataset = torch.load(mini_dataset_path)
        #print(mini_dataset)
        data, labels = mini_dataset.get_tensor_dataset()
        data = data.cpu().numpy()
        labels = labels.numpy()

        ds_accs = []
        for round in range(20):
            # measure cross-validation accuracy using a LinearSVC over sequence embedding
            model = LinearSVC(max_iter=2000)    
            results = cross_validate(
                estimator=model, 
                X=data,
                y=labels,
                scoring='accuracy',
                cv=5,
                return_estimator=True
            )
            acc = np.mean(results['test_score'])
            #print("Round %d, cross-val acc %f" % (round, acc))
            ds_accs.append(acc.item())

            # 5 x num_features
            weights = np.concatenate([est.coef_ for est in results['estimator']])

            # for each weight, compute its absolute max value across models
            # then take the top 20 maximum weights to indicate the most
            # important features
            # num_features
            max_weights = np.max(np.abs(weights), axis=0)
            # feature indices from most important to least important
            # num_features
            features_idx = np.argsort(max_weights)[::-1]

            # eliminate features from dataset
            data = np.delete(data, features_idx[:20], 1)
            #print("   data shape: ", data.shape)

        #print("label = ", mini_dataset.pair_label, ", acc = ", ds_accs)
        test_curves.append(ds_accs)
        test_labels.append(mini_dataset.pair_label)
        #if idx == 5:
        #    break

Processed 0 test mini-datasets 
Processed 1 test mini-datasets 
Processed 2 test mini-datasets 
Processed 3 test mini-datasets 
Processed 4 test mini-datasets 
Processed 5 test mini-datasets 
Processed 6 test mini-datasets 
Processed 7 test mini-datasets 
Processed 8 test mini-datasets 
Processed 9 test mini-datasets 
Processed 10 test mini-datasets 
Processed 11 test mini-datasets 
Processed 12 test mini-datasets 
Processed 13 test mini-datasets 
Processed 14 test mini-datasets 
Processed 15 test mini-datasets 
Processed 16 test mini-datasets 
Processed 17 test mini-datasets 
Processed 18 test mini-datasets 
Processed 19 test mini-datasets 
Processed 20 test mini-datasets 
Processed 21 test mini-datasets 
Processed 22 test mini-datasets 
Processed 23 test mini-datasets 
Processed 24 test mini-datasets 
Processed 25 test mini-datasets 
Processed 26 test mini-datasets 
Processed 27 test mini-datasets 
Processed 28 test mini-datasets 
Processed 29 test mini-datasets 
Processed 30 test mi

Processed 245 test mini-datasets 
Processed 246 test mini-datasets 
Processed 247 test mini-datasets 
Processed 248 test mini-datasets 
Processed 249 test mini-datasets 
Processed 250 test mini-datasets 
Processed 251 test mini-datasets 
Processed 252 test mini-datasets 
Processed 253 test mini-datasets 
Processed 254 test mini-datasets 
Processed 255 test mini-datasets 
Processed 256 test mini-datasets 
Processed 257 test mini-datasets 
Processed 258 test mini-datasets 
Processed 259 test mini-datasets 
Processed 260 test mini-datasets 
Processed 261 test mini-datasets 
Processed 262 test mini-datasets 
Processed 263 test mini-datasets 
Processed 264 test mini-datasets 
Processed 265 test mini-datasets 
Processed 266 test mini-datasets 
Processed 267 test mini-datasets 
Processed 268 test mini-datasets 
Processed 269 test mini-datasets 
Processed 270 test mini-datasets 
Processed 271 test mini-datasets 
Processed 272 test mini-datasets 
Processed 273 test mini-datasets 
Processed 274 

Processed 486 test mini-datasets 
Processed 487 test mini-datasets 
Processed 488 test mini-datasets 
Processed 489 test mini-datasets 
Processed 490 test mini-datasets 
Processed 491 test mini-datasets 
Processed 492 test mini-datasets 
Processed 493 test mini-datasets 
Processed 494 test mini-datasets 
Processed 495 test mini-datasets 
Processed 496 test mini-datasets 
Processed 497 test mini-datasets 
Processed 498 test mini-datasets 
Processed 499 test mini-datasets 
Processed 500 test mini-datasets 
Processed 501 test mini-datasets 
Processed 502 test mini-datasets 
Processed 503 test mini-datasets 
Processed 504 test mini-datasets 
Processed 505 test mini-datasets 
Processed 506 test mini-datasets 
Processed 507 test mini-datasets 
Processed 508 test mini-datasets 
Processed 509 test mini-datasets 
Processed 510 test mini-datasets 
Processed 511 test mini-datasets 
Processed 512 test mini-datasets 
Processed 513 test mini-datasets 
Processed 514 test mini-datasets 
Processed 515 

Processed 727 test mini-datasets 
Processed 728 test mini-datasets 
Processed 729 test mini-datasets 
Processed 730 test mini-datasets 
Processed 731 test mini-datasets 
Processed 732 test mini-datasets 
Processed 733 test mini-datasets 
Processed 734 test mini-datasets 
Processed 735 test mini-datasets 
Processed 736 test mini-datasets 
Processed 737 test mini-datasets 
Processed 738 test mini-datasets 
Processed 739 test mini-datasets 
Processed 740 test mini-datasets 
Processed 741 test mini-datasets 
Processed 742 test mini-datasets 
Processed 743 test mini-datasets 
Processed 744 test mini-datasets 
Processed 745 test mini-datasets 
Processed 746 test mini-datasets 
Processed 747 test mini-datasets 
Processed 748 test mini-datasets 
Processed 749 test mini-datasets 
Processed 750 test mini-datasets 
Processed 751 test mini-datasets 
Processed 752 test mini-datasets 
Processed 753 test mini-datasets 
Processed 754 test mini-datasets 
Processed 755 test mini-datasets 
Processed 756 

Processed 968 test mini-datasets 
Processed 969 test mini-datasets 
Processed 970 test mini-datasets 
Processed 971 test mini-datasets 
Processed 972 test mini-datasets 
Processed 973 test mini-datasets 
Processed 974 test mini-datasets 
Processed 975 test mini-datasets 
Processed 976 test mini-datasets 
Processed 977 test mini-datasets 
Processed 978 test mini-datasets 
Processed 979 test mini-datasets 
Processed 980 test mini-datasets 
Processed 981 test mini-datasets 
Processed 982 test mini-datasets 
Processed 983 test mini-datasets 
Processed 984 test mini-datasets 
Processed 985 test mini-datasets 
Processed 986 test mini-datasets 
Processed 987 test mini-datasets 
Processed 988 test mini-datasets 
Processed 989 test mini-datasets 
Processed 990 test mini-datasets 
Processed 991 test mini-datasets 
Processed 992 test mini-datasets 
Processed 993 test mini-datasets 
Processed 994 test mini-datasets 
Processed 995 test mini-datasets 
Processed 996 test mini-datasets 
Processed 997 

...

Processed 1673 test mini-datasets 
Processed 1674 test mini-datasets 
Processed 1675 test mini-datasets 
Processed 1676 test mini-datasets 
Processed 1677 test mini-datasets 
Processed 1678 test mini-datasets 
Processed 1679 test mini-datasets 
Processed 1680 test mini-datasets 
Processed 1681 test mini-datasets 
Processed 1682 test mini-datasets 
Processed 1683 test mini-datasets 
Processed 1684 test mini-datasets 
Processed 1685 test mini-datasets 
Processed 1686 test mini-datasets 
Processed 1687 test mini-datasets 
Processed 1688 test mini-datasets 
Processed 1689 test mini-datasets 
Processed 1690 test mini-datasets 
Processed 1691 test mini-datasets 
Processed 1692 test mini-datasets 
Processed 1693 test mini-datasets 
Processed 1694 test mini-datasets 
Processed 1695 test mini-datasets 
Processed 1696 test mini-datasets 
Processed 1697 test mini-datasets 
Processed 1698 test mini-datasets 
Processed 1699 test mini-datasets 
Processed 1700 test mini-datasets 
Processed 1701 test 

Processed 1908 test mini-datasets 
Processed 1909 test mini-datasets 
Processed 1910 test mini-datasets 
Processed 1911 test mini-datasets 
Processed 1912 test mini-datasets 
Processed 1913 test mini-datasets 
Processed 1914 test mini-datasets 
Processed 1915 test mini-datasets 
Processed 1916 test mini-datasets 
Processed 1917 test mini-datasets 
Processed 1918 test mini-datasets 
Processed 1919 test mini-datasets 
Processed 1920 test mini-datasets 
Processed 1921 test mini-datasets 
Processed 1922 test mini-datasets 
Processed 1923 test mini-datasets 
Processed 1924 test mini-datasets 
Processed 1925 test mini-datasets 
Processed 1926 test mini-datasets 
Processed 1927 test mini-datasets 
Processed 1928 test mini-datasets 
Processed 1929 test mini-datasets 
Processed 1930 test mini-datasets 
Processed 1931 test mini-datasets 
Processed 1932 test mini-datasets 
Processed 1933 test mini-datasets 
Processed 1934 test mini-datasets 
Processed 1935 test mini-datasets 
Processed 1936 test 

Processed 2143 test mini-datasets 
Processed 2144 test mini-datasets 
Processed 2145 test mini-datasets 
Processed 2146 test mini-datasets 
Processed 2147 test mini-datasets 
Processed 2148 test mini-datasets 
Processed 2149 test mini-datasets 
Processed 2150 test mini-datasets 
Processed 2151 test mini-datasets 
Processed 2152 test mini-datasets 
Processed 2153 test mini-datasets 
Processed 2154 test mini-datasets 
Processed 2155 test mini-datasets 
Processed 2156 test mini-datasets 
Processed 2157 test mini-datasets 
Processed 2158 test mini-datasets 
Processed 2159 test mini-datasets 
Processed 2160 test mini-datasets 
Processed 2161 test mini-datasets 
Processed 2162 test mini-datasets 
Processed 2163 test mini-datasets 
Processed 2164 test mini-datasets 
Processed 2165 test mini-datasets 
Processed 2166 test mini-datasets 
Processed 2167 test mini-datasets 
Processed 2168 test mini-datasets 
Processed 2169 test mini-datasets 
Processed 2170 test mini-datasets 
Processed 2171 test 

Processed 2378 test mini-datasets 
Processed 2379 test mini-datasets 
Processed 2380 test mini-datasets 
Processed 2381 test mini-datasets 
Processed 2382 test mini-datasets 
Processed 2383 test mini-datasets 
Processed 2384 test mini-datasets 
Processed 2385 test mini-datasets 
Processed 2386 test mini-datasets 
Processed 2387 test mini-datasets 
Processed 2388 test mini-datasets 
Processed 2389 test mini-datasets 
Processed 2390 test mini-datasets 
Processed 2391 test mini-datasets 
Processed 2392 test mini-datasets 
Processed 2393 test mini-datasets 
Processed 2394 test mini-datasets 
Processed 2395 test mini-datasets 
Processed 2396 test mini-datasets 
Processed 2397 test mini-datasets 
Processed 2398 test mini-datasets 
Processed 2399 test mini-datasets 
Processed 2400 test mini-datasets 
Processed 2401 test mini-datasets 
Processed 2402 test mini-datasets 
Processed 2403 test mini-datasets 
Processed 2404 test mini-datasets 
Processed 2405 test mini-datasets 
Processed 2406 test 

Processed 2613 test mini-datasets 
Processed 2614 test mini-datasets 
Processed 2615 test mini-datasets 
Processed 2616 test mini-datasets 
Processed 2617 test mini-datasets 
Processed 2618 test mini-datasets 
Processed 2619 test mini-datasets 
Processed 2620 test mini-datasets 
Processed 2621 test mini-datasets 
Processed 2622 test mini-datasets 
Processed 2623 test mini-datasets 
Processed 2624 test mini-datasets 
Processed 2625 test mini-datasets 
Processed 2626 test mini-datasets 
Processed 2627 test mini-datasets 
Processed 2628 test mini-datasets 
Processed 2629 test mini-datasets 


In [104]:
# save test curves
with open('test_curves.pickle', 'wb') as fp:
    pickle.dump((np.array(test_curves), np.array(test_labels)), fp)

In [137]:
# train meta-classifier on features from train curves
all_features_train = np.concatenate((
    train_curves_np,
    np.gradient(train_curves_np, axis=1), # gradient
    np.diff(train_curves_np),             # difference
    np.diff(train_curves_np, 2)           # 2nd order difference
), axis=1)
model = LinearSVC(max_iter=2000)
#model = SVC(kernel='linear', max_iter=2000)
model.fit(all_features_train, train_labels_np)

LinearSVC(max_iter=2000)

In [138]:
# test meta-classifier on features from test curves
test_curves_np = np.array(test_curves)
test_labels_np = np.array(test_labels)
all_features_test = np.concatenate((
    test_curves_np,
    np.gradient(test_curves_np, axis=1), # gradient
    np.diff(test_curves_np),             # difference
    np.diff(test_curves_np, 2)           # 2nd order difference
), axis=1)

In [139]:
decision_scores = model.decision_function(all_features_test)
fpr, tpr, thres = roc_curve(test_labels_np, decision_scores)
print('AUC: {:.3f}'.format(roc_auc_score(test_labels_np, decision_scores)))

AUC: 0.857


array([[ 0.78969218,  0.13286236,  0.76403936,  0.20602371, -0.65682982,
        -0.01282641,  0.03658067, -0.55801565],
       [ 0.31807901,  0.55488952,  0.82720848,  0.0398528 ,  0.23681051,
         0.25456473, -0.25751836, -0.78735568],
       [ 0.27714685,  0.50222159,  0.02689236,  0.22334613,  0.22507474,
        -0.12512724, -0.13943773,  0.19645377]])